In [1]:
import csv
from pprint import pprint
from time import sleep

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, InvalidArgumentException

import re
import json

In [2]:
options = Options()
options.add_argument('--disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Chrome("../chromedriver/chromedriver.exe", options=options)

C:\Users\user\AppData\Local\Temp\ipykernel_21896\2482000279.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome("../chromedriver/chromedriver.exe", options=options)


In [3]:
driver.get("view-source:https://www.tokopedia.com/search?navsource=&page=2&q=rx%206600&rf=true&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&st=product")
driver.implicitly_wait(30)

In [4]:
source_page = driver.page_source

In [5]:
raw_data_tokped = re.search('window.__cache=(.*)};', source_page)
raw_data_tokped = raw_data_tokped.group(1) + "}"

In [6]:
json_data = json.loads(raw_data_tokped)

In [7]:
dict_keys = [key for key in json_data if key.startswith("TopAdsProduct") or key.startswith("AceSearchUnifyProduct")  if "." not in key]
len(dict_keys)

80

In [8]:
def extract_data_from_json(json_data, key):
    title = json_data[key].get("name")
    price = json_data[key].get("price")
    if price is None:
        price = json_data[key].get("price_format")
    return {
        "title": title,
        "price": price
    }
extracted_data = [extract_data_from_json(json_data, key) for key in dict_keys]
extracted_data

[{'title': 'SAPPHIRE PULSE RADEON RX 6600 GAMING 8GB GDDR6 HDMI/TRIPLE DP RX6600',
  'price': 'Rp5.099.000'},
 {'title': 'ASROCK AMD Radeon RX 6600 Challenger D 8GB GDDR6 - RX6600 CLD 8G',
  'price': 'Rp5.550.000'},
 {'title': 'ASROCK RX 6600 XT Challenger D OC 8GB GDDR6',
  'price': 'Rp5.990.000'},
 {'title': 'ASUS DUAL RX 6600 XT OC 8Gb GDDR6', 'price': 'Rp7.720.000'},
 {'title': 'POWERCOLOR HELLHOUND AMD RADEON RX 6600 8GB GDDR6',
  'price': 'Rp4.890.000'},
 {'title': 'Gigabyte VGA Radeon™ RX 6600 XT EAGLE 8G (GV-R66XTEAGLE-8GD)',
  'price': 'Rp7.896.000'},
 {'title': 'VGA AMD SAPPHIRE RX 6600 / RX6600 8GB PULSE',
  'price': 'Rp5.999.999'},
 {'title': 'GIGABYTE RADEON RX 6600 XT GAMING OC PRO 8G GDDR6 128-BIT',
  'price': 'Rp7.590.000'},
 {'title': 'VGA MSI Radeon RX 6600 XT RX6600XT MECH 2X 8GB GDDR6 OC 128-bit',
  'price': 'Rp9.899.000'},
 {'title': 'VGA ASROCK RX 6600 CHALLENGER 8GB DDR6 RX6600',
  'price': 'Rp5.500.000'},
 {'title': 'VGA ASROCK AMD RADEON RX 6600 XT PHANTOM GAMI

In [6]:
searchbar = driver.find_element("xpath", '//*[@id="search-container"]/form/div/div/div/input')
searchbar.click()
searchbar.send_keys("tv sony")
searchbar.send_keys(Keys.ENTER)
print("search for {}..".format("tv sony"))

In [5]:
def scroll_to_bottom():
    for i in range(7):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        sleep(0.7)

In [6]:
try:
    overlay = driver.find_element_by_xpath("/html/body/div[5]/div[3]")
    /html/body/div[5]/div[6]
    overlay.click()
    scroll_to_bottom()
    driver.find_element_by_xpath("//*[contains(text(),'Oke, Lanjut Belanja')]").click()
except (NoSuchElementException, TimeoutException):
    pass

In [7]:
urls = driver.find_elements_by_xpath('//*[@id="zeus-root"]/div/div[2]/div/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/a')
urls = [url.get_attribute('href') for url in urls]
promo_url = []

for url in urls[:]:
    if 'ta.tokopedia' in url:
        promo_url.append(url)
        urls.remove(url)
        
        

In [8]:
driver.get(urls[0])

In [14]:
separator = 'https%3A%2F%2F'
for url in promo_url:
    url = url.split(separator)
    url = url[1].replace("%2F", "/")
    urls.append(url)

len(urls)

80

In [19]:
title = driver.find_element_by_xpath('//*[@id="pdp_comp-product_content"]/div/h1').text
sold = driver.find_element_by_xpath('//*[@id="pdp_comp-product_content"]/div/div[1]/div/div[1]').text
price = driver.find_element_by_xpath('//*[@id="pdp_comp-product_content"]/div/div[2]/div').text
city = driver.find_element_by_xpath('//*[@id="pdp_comp-shipment"]/div/div/div[1]/div/b').text

result = {
    'title': title,
    'price': price,
    'sold': sold,
    'city': city
}
print(result)

{'title': 'SONY LED TV KD 85X8000H - SMART TV LED 85 INCH ANDROID TV 4K 85X8000', 'price': 'Rp41.999.000', 'sold': 'Terjual 34', 'city': 'Kota Tangerang Selatan'}
